# 🚀 Day 1-3:  SVR (Support Vector Regression)

이번 세션에서는 회귀 문제에 강력한 성능을 보이는 알고리즘 중 하나인 **서포트 벡터 회귀 (Support Vector Regression, SVR)** 를 학습합니다. 

특히 비선형 데이터를 다루는 데 효과적인 **RBF 커널** 을 중심으로 SVR의 원리를 파헤치고,

`scikit-learn`을 활용한 실습을 통해 개념을 코드로 체화하는 시간을 갖겠습니다.

---

### 1. SVR 이란 무엇일까요?

선형 회귀(Linear Regression)는 모든 데이터 샘플과의 오차(잔차) 제곱의 합을 최소화하는 직선을 찾는 것이 목표였습니다. 

데이터 포인트 하나하나가 모델의 학습에 직접적인 영향을 주죠.

반면, SVR은 조금 다른 철학을 가집니다. **"최대한 많은 데이터 샘플이 하나의 '도로(tube)' 안에 포함되도록 경계선을 찾자!"** 는 아이디어에서 출발합니다. 

이 도로의 폭을 결정하는 것이 바로 SVR의 핵심 파라미터인 **엡실론($\epsilon$)** 입니다.

  * **SVR의 목표**: 특정 너비($\epsilon$)를 가진 '튜브' 내에 오차를 허용하면서, 이 튜브에서 벗어나는 샘플의 오류만 최소화하는 최적의 회귀선을 찾는 것.
  
  * **Support Vectors**: 이 '튜브'의 경계선이나 그 바깥에 위치하여 모델(회귀선)을 결정하는 데 영향을 주는 중요한 데이터 포인트들입니다. 이 서포트 벡터들 덕분에 SVR은 다른 데이터 포인트의 노이즈에 비교적 강인한(robust) 모델을 만들 수 있습니다.

![SVR](https://www.mdpi.com/buildings/buildings-12-01792/article_deploy/html/images/buildings-12-01792-g003-550.jpg)

*(위 그림에서 실선이 회귀선, 점선이 $\epsilon$-tube의 경계, 별표로 표시된 점들이 서포트 벡터입니다.)*


-----

### 2. 핵심 개념 1: 엡실론 인센서티브 튜브 ($\epsilon$-Insensitive Tube)

SVR의 가장 독특하고 중요한 개념은 바로 **엡실론($\epsilon$)** 입니다. 

'Epsilon-Insensitive'라는 이름에서 알 수 있듯, SVR은 예측값과 실제값의 차이(오차)가 $\epsilon$보다 작으면 그 오차는 **"문제 삼지 않겠다(insensitive)"**, 즉 무시합니다.

  * **$|y\_i - f(x\_i)| \leq \epsilon$**: 오차가 $\epsilon$ 이내라면, 모델의 비용(손실)은 0입니다.
  * **$|y\_i - f(x\_i)| > \epsilon$**: 오차가 $\epsilon$을 초과하면, 그 초과분에 대해서만 비용을 계산합니다.

이 $\epsilon$ 값은 우리가 직접 설정해야 하는 하이퍼파라미터입니다.

  * **`epsilon` 값이 크면?**: 더 넓은 튜브(도로)가 만들어집니다. 모델은 더 많은 오차를 용납하게 되므로, 모델이 단순해지는 경향(underfitting)이 있을 수 있습니다.
  * **`epsilon` 값이 작으면?**: 더 좁은 튜브가 만들어집니다. 아주 작은 오차도 용납하지 않으려 하므로, 모델이 데이터의 미세한 패턴까지 학습하려다 노이즈에 과하게 반응(overfitting)할 수 있습니다.


#### 🧠 코드로 이해하기: $\epsilon$ 값에 따른 튜브의 변화

간단한 1차원 데이터를 만들고, $\epsilon$ 값을 다르게 설정했을 때 SVR 모델과 튜브가 어떻게 변하는지 시각화해 보겠습니다.

In [2]:
import numpy as np
import plotly.graph_objects as go
from sklearn.svm import SVR

# 1. 샘플 데이터 생성
np.random.seed(42)
X = np.sort(5 * np.random.rand(40, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - np.random.rand(8)) # 노이즈 추가

In [9]:
# 2. Epsilon 값에 따른 SVR 모델 학습 

# Epsilon 값별로 개별 모델 생성 및 학습
svr_01 = SVR(kernel='rbf', C=1e3, gamma=0.1, epsilon=0.1)
svr_05 = SVR(kernel='rbf', C=1e3, gamma=0.1, epsilon=0.5)
svr_10 = SVR(kernel='rbf', C=1e3, gamma=0.1, epsilon=1.0)

# 각 모델 학습
svr_01.fit(X, y)
svr_05.fit(X, y)
svr_10.fit(X, y)

# 각 모델의 예측값 생성
y_pred_01 = svr_01.predict(X)
y_pred_05 = svr_05.predict(X)
y_pred_10 = svr_10.predict(X)

In [10]:
# 3. 시각화
from plotly.subplots import make_subplots

# 서브플롯 생성 (1행 3열)
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=['Epsilon = 0.1', 'Epsilon = 0.5', 'Epsilon = 1.0'],
                    horizontal_spacing=0.1)

# 모델과 예측값 리스트
models = [svr_01, svr_05, svr_10]
predictions = [y_pred_01, y_pred_05, y_pred_10]
epsilons = [0.1, 0.5, 1.0]
colors = ['blue', 'green', 'purple']

# 각 서브플롯 생성
for i, (model, pred, eps, color) in enumerate(zip(models, predictions, epsilons, colors)):
    col = i + 1
    
    # 데이터 포인트
    fig.add_trace(go.Scatter(x=X.flatten(), y=y, mode='markers', 
                             name=f'Data (eps={eps})',
                             marker=dict(color='black', size=6),
                             showlegend=False), row=1, col=col)
    
    # SVR 예측선
    fig.add_trace(go.Scatter(x=X.flatten(), y=pred, mode='lines',
                             line=dict(color=color, width=3),
                             name=f'SVR (eps={eps})',
                             showlegend=False), row=1, col=col)
    
    # 하단 튜브 경계
    fig.add_trace(go.Scatter(x=X.flatten(), y=pred - eps, mode='lines',
                             line=dict(color=color, width=1, dash='dash'),
                             name=f'Tube boundary (eps={eps})',
                             showlegend=False), row=1, col=col)
    
    # 상단 튜브 경계 (fill로 튜브 영역 표시)
    fig.add_trace(go.Scatter(x=X.flatten(), y=pred + eps, mode='lines',
                             line=dict(color=color, width=1, dash='dash'),
                             fill='tonexty',
                             fillcolor=f'rgba({255 if color=="blue" else 0 if color=="green" else 128},{128 if color=="green" else 0},{255 if color=="blue" else 128 if color=="purple" else 0}, 0.2)',
                             name=f'Epsilon tube (eps={eps})',
                             showlegend=False), row=1, col=col)

fig.update_layout(
    title_text='Epsilon 값에 따른 SVR Tube 변화',
    height=400,
    template='plotly_white',
    showlegend=False
)

# 각 서브플롯의 축 레이블 설정
for i in range(1, 4):
    fig.update_xaxes(title_text='Feature (X)', row=1, col=i)
    fig.update_yaxes(title_text='Target (y)', row=1, col=1)

fig.show()

#### 🎯 연습문제 1

SVR 모델을 학습시킬 때, 우리가 예측하려는 타겟 값의 스케일(예: 0\~1 사이인지, 100만\~1000만 사이인지)을 고려한다면, `epsilon` 값은 어떻게 설정하는 것이 합리적일까요? 만약 주택 가격(억 단위)을 예측하는데 `epsilon=0.1`로 설정한다면 어떤 문제가 발생할 수 있을지 설명해 보세요.

> 설명을 작성해보세요.


-----

### 3. 핵심 개념 2: RBF 커널과 하이퍼파라미터 (C, $\gamma$)

SVR이 비선형 데이터 패턴을 학습할 수 있게 만드는 비장의 무기가 바로 **커널 트릭(Kernel Trick)** 입니다. 

그중에서도 **RBF(Radial Basis Function) 커널** 은 가장 널리 쓰이며 강력한 성능을 자랑합니다.

RBF 커널은 복잡하게 얽힌 저차원 데이터를, 선형적으로 분리(또는 회귀) 가능한 고차원 공간으로 '매핑'해주는 역할을 합니다. 

우리는 그 고차원 공간을 직접 계산할 필요 없이, 커널 함수를 통해 그 결과를 얻을 수 있습니다.

RBF 커널을 사용할 때 가장 중요한 두 가지 하이퍼파라미터는 **`C`** 와 **`gamma`($\gamma$)** 입니다.

#### ⚙️ `C` (Cost, 비용 매개변수)

`C`는 **모델의 복잡도와 오차 허용 범위 사이의 트레이드오프** 를 조절합니다. 

$\epsilon$-tube 바깥에 존재하는 데이터 포인트(margin violation)에 대해 얼마나 큰 페널티를 부여할지를 결정합니다.

  * **`C` 값이 크면? (Hard Margin)**: 페널티가 커집니다. 모델은 튜브 바깥의 데이터 포인트를 용납하지 않으려 매우 노력합니다. 이는 훈련 데이터에 거의 완벽하게 들어맞는 복잡한 모델을 만들 수 있지만, 새로운 데이터에 대해서는 성능이 떨어지는 **과적합(Overfitting)** 의 위험이 커집니다.
  
  * **`C` 값이 작으면? (Soft Margin)**: 페널티가 작아집니다. 모델은 어느 정도의 오차를 너그럽게 허용하며, 더 단순하고 일반적인 패턴을 학습하려 합니다. 이는 **과소적합(Underfitting)** 으로 이어질 수 있습니다.

#### ⚙️ `gamma` ($\gamma$)

`gamma`는 **하나의 훈련 데이터 포인트가 미치는 영향력의 범위** 를 결정합니다. RBF 커널의 모양을 조절하는 파라미터라고 생각할 수 있습니다.

  * **`gamma` 값이 크면?**: 영향력의 범위가 좁아집니다. 각 데이터 포인트는 자신과 매우 가까운 주변에만 영향을 미칩니다. 결정 경계(회귀선)는 매우 구불구불하고 복잡해지며, 훈련 데이터의 노이즈까지 민감하게 학습하여 **과적합(Overfitting)** 될 가능성이 높습니다.
  
  * **`gamma` 값이 작으면?**: 영향력의 범위가 넓어집니다. 각 데이터 포인트는 멀리 있는 다른 데이터 포인트에도 영향을 줍니다. 결정 경계는 매우 부드럽고 단순한 형태를 띠게 되어, **과소적합(Underfitting)** 이 발생할 수 있습니다.


#### 🧠 코드로 이해하기: `C`와 `gamma`의 영향력 시각화

`C`와 `gamma` 값의 변화가 SVR 모델의 회귀선을 어떻게 바꾸는지 직접 확인해 봅시다.

In [11]:
import pandas as pd
import plotly.express as px
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# 1. 서울시 따릉이 데이터 일부 사용
# 실습의 편의를 위해 시간(Hour)과 대여량(Rented Bike Count)만 사용
# 원본 데이터 URL: https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv'
df = pd.read_csv(url, encoding='ISO-8859-1')

# 컬럼명 정리
df.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
              'Wind_speed', 'Visibility', 'Dew_point_temp', 'Solar_Radiation',
              'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']

# 간단한 전처리
df_sample = df[['Hour', 'Rented_Bike_Count']].groupby('Hour').mean().reset_index()

X_sample = df_sample[['Hour']].values
y_sample = df_sample['Rented_Bike_Count'].values

# 데이터 스케일링 (SVR은 스케일링이 중요!)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X_sample)
y_scaled = scaler_y.fit_transform(y_sample.reshape(-1, 1)).ravel()


# 2. C와 gamma 값에 따른 SVR 모델 비교
C_values = [0.1, 1, 100]
gamma_values = [0.01, 1, 100]

results = []

for C in C_values:
    for gamma in gamma_values:
        model = SVR(kernel='rbf', C=C, gamma=gamma)
        model.fit(X_scaled, y_scaled)

        # 예측값을 원래 스케일로 복원
        y_pred_scaled = model.predict(X_scaled)
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))

        # 결과를 DataFrame에 저장
        temp_df = pd.DataFrame({
            'Hour': X_sample.flatten(),
            'Predicted_Count': y_pred.flatten(),
            'C': C,
            'gamma': gamma
        })
        results.append(temp_df)

results_df = pd.concat(results)

# 3. Plotly로 시각화
fig = px.line(results_df, x='Hour', y='Predicted_Count',
              facet_col='C', facet_row='gamma',
              category_orders={"gamma": gamma_values, "C": C_values},
              labels={'Predicted_Count': '예측 대여량', 'Hour': '시간', 'gamma': 'Gamma', 'C': 'C 값'},
              title='C와 Gamma 값에 따른 SVR(RBF) 회귀선 변화')

# 원본 데이터 추가
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[-1])) # 범례 정리
fig.add_trace(go.Scatter(x=X_sample.flatten(), y=y_sample, mode='markers', name='Actual', marker=dict(color='grey', opacity=0.6)))

fig.show()

위 시각화 결과를 통해 `C`와 `gamma` 값이 어떻게 상호작용하며 모델의 유연성을 결정하는지 직관적으로 파악할 수 있습니다.

  * **과소적합 영역 (좌측 상단)**: `C`와 `gamma`가 모두 작으면 모델이 너무 단순해져 데이터의 패턴을 거의 학습하지 못합니다.
  * **과적합 영역 (우측 하단)**: `C`와 `gamma`가 모두 크면 모델이 훈련 데이터에 과도하게 맞춰져 구불구불한 형태가 됩니다.
  * **적절한 모델 영역 (중앙)**: 적절한 `C`와 `gamma` 값을 통해 데이터의 전반적인 추세를 잘 따르는 부드러운 회귀선을 얻을 수 있습니다.

#### 🎯 연습문제 2

`scikit-learn`의 `SVR`에서 `gamma`의 기본값은 `'scale'`입니다. 

이 `'scale'` 옵션은 `1 / (n_features * X.var())`로 계산됩니다. 

왜 `gamma` 값을 데이터의 분산(variance)에 반비례하도록 설정하는 것이 합리적인 선택일지, 데이터 스케일의 관점에서 설명해 보세요. 

(힌트: 피처의 값 범위가 매우 크거나 작을 때를 생각해 보세요.)

> 설명을 작성해보세요.